In [1]:
# Import dependices.
import iris
from iris.coords import AuxCoord
from iris.cube import CubeList
import xarray as xr
from tqdm import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM2D, Dense, TimeDistributed
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
import tensorflow as tf
import numpy as np
import sys

/opt/conda/lib/python3.7/site-packages/xarray/core/merge.py:10: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  PANDAS_TYPES = (pd.Series, pd.DataFrame, pd.Panel)


In [2]:
# Ensure reproducibility.
tf.random.set_seed(13)

In [3]:
# Function to split data into windows.
def window_data(dataset, past_history, future_target):
    X, y = [], []
    
    for i in range(dataset.shape[0]):
        # Find the end.
        end_ix = i + past_history
        out_end_ix = end_ix + future_target
        
        # Determine if we are beyond the dataset.
        if out_end_ix > dataset.shape[0]:
            break
        
        # Gather the input and output components.
        seq_x, seq_y = dataset[i:end_ix], dataset[end_ix:out_end_ix]

        # Append to list.
        X.append(seq_x)
        y.append(seq_y)

    return X, y

In [4]:
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, ds, past_history, batch_size=32, shuffle=True, load=True, mean=None, std=None):
        """
        Template from https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly
        
        Args:
            ds: Dataset containing all variables
            past_history: how much of the past in hours is shown to the model
            batch_size: Batch size
            shuffle: bool. If True, data is shuffled.
            load: bool. If True, datadet is loaded into RAM.
            mean: If None, compute mean from data.
            std: If None, compute standard deviation from data.
        """
        self.ds = ds
        self.batch_size = batch_size
        self.shuffle = shuffle

        data = []
        generic_level = xr.DataArray([1], coords={'level': [1]}, dims=['level'])
        data.append(ds.expand_dims({'level': generic_level}, 1))

        self.data = xr.concat(data, 'level').transpose('time', 'latitude', 'longitude', 'level')
        self.mean = self.data.mean(('time', 'latitude', 'longitude')).compute() if mean is None else mean
        self.std = self.data.std('time').mean(('latitude', 'longitude')).compute() if std is None else std
        # Normalize
        self.data = (self.data - self.mean) / self.std
        
        # Window data.
        self.window = window_data(self.data, past_history, 1)

        self.n_samples = len(self.window[0])
        self.on_epoch_end()

        # For some weird reason calling .load() earlier messes up the mean and std computations
        if load: print('Loading data into RAM'); self.data.load()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.ceil(self.n_samples / self.batch_size))

    def __getitem__(self, i):
        'Generate one batch of data'
        idxs = self.idxs[i * self.batch_size:(i + 1) * self.batch_size]

        X, y = self.window[0], self.window[1]
        X = np.array([X[k].values for k in idxs])
        y = np.array([y[k].values for k in idxs])
        
        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.idxs = np.arange(self.n_samples)
        if self.shuffle == True:
            np.random.shuffle(self.idxs)

In [5]:
# Define root mean squared error loss function
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true))) 

In [6]:
# Load data.
folder = 'dataset/'

# Time steps.
nsteps = 600

# Parameter label.
parameter_label = "2m_temperature"

# Load dataset.
dataset = iris.load(folder + "/" + parameter_label + '/*.nc')[0][:nsteps]

# Convert cube to data array.
dataset = xr.DataArray.from_iris(dataset)

In [7]:
def model(data, past_history, epochs, bs):
    # Define number of latitude and longitude points.
    nlat, nlon = data.shape[1], data.shape[2]
    
    # Define training dataset and validation dataset.
    train_dataset = data[:int(0.7 * data.shape[0])]
    val_dataset = data[int(0.7 * data.shape[0]):int(0.9 * data.shape[0])]
    
    # Training dataset.
    train_generator = DataGenerator(
        train_dataset, 
        past_history, 
        batch_size=bs, 
        load=False
    )
    
    # Validation dataset.
    val_generator = DataGenerator(
        val_dataset, 
        past_history, 
        batch_size=bs, 
        mean=train_generator.mean, 
        std=train_generator.std,
        load=False,
        shuffle=False
    )

    # Create, and train models.
    # Optimiser.
    opt = Adam(lr=1e-3, decay=1e-5)
    # Create model.
    model = Sequential()
    
    # First layer.
    model.add(
        ConvLSTM2D(
            filters=64, 
            kernel_size=(7, 7),
            input_shape=(past_history, nlat, nlon, 1), 
            padding='same', 
            return_sequences=True, 
            activation='tanh', 
            recurrent_activation='hard_sigmoid',
            kernel_initializer='glorot_uniform', 
            unit_forget_bias=True, 
            dropout=0.3, 
            recurrent_dropout=0.3, 
            go_backwards=True
        )
    )
    model.add(BatchNormalization())
    
    # Second layer.
    model.add(
        ConvLSTM2D(
            filters=32, 
            kernel_size=(7, 7), 
            padding='same', 
            return_sequences=True, 
            activation='tanh', 
            recurrent_activation='hard_sigmoid', 
            kernel_initializer='glorot_uniform', 
            unit_forget_bias=True, 
            dropout=0.4, 
            recurrent_dropout=0.3, 
            go_backwards=True
        )
    )
    model.add(BatchNormalization())
    
    # Third layer.
    model.add(
        ConvLSTM2D(
            filters=32, 
            kernel_size=(7, 7), 
            padding='same', 
            return_sequences=True, 
            activation='tanh', 
            recurrent_activation='hard_sigmoid', 
            kernel_initializer='glorot_uniform', 
            unit_forget_bias=True, 
            dropout=0.4, 
            recurrent_dropout=0.3, 
            go_backwards=True
        )
    )
    model.add(BatchNormalization())
    
    # Final layer.
    model.add(
        ConvLSTM2D(
            filters=32, 
            kernel_size=(7, 7), 
            padding='same', 
            return_sequences=False, 
            activation='tanh', 
            recurrent_activation='hard_sigmoid', 
            kernel_initializer='glorot_uniform', 
            unit_forget_bias=True, 
            dropout=0.4, 
            recurrent_dropout=0.3, 
            go_backwards=True
        )
    )
    model.add(BatchNormalization())
    
    # Add dense layer.
    model.add(Dense(1)) 
    
    # Compile model.
    model.compile(
        optimizer=opt, loss=rmse, metrics=['mean_absolute_error', 'mse']
    )
    
    # Summary of model.
    model.summary()

    # Train.
    model.fit(
        train_generator, 
        validation_data=val_generator,
        epochs=epochs,
        callbacks=[
            tf.keras.callbacks.EarlyStopping(
                monitor='val_loss',
                min_delta=0,
                patience=2, 
                mode='auto'
            )
        ],
    )
    
    return model

In [ ]:
# Define model.
model = model(dataset, (15*12), epochs=100, bs=32)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d (ConvLSTM2D)    (None, 84, 37, 72, 64)    815616    
_________________________________________________________________
batch_normalization (BatchNo (None, 84, 37, 72, 64)    256       
_________________________________________________________________
conv_lst_m2d_1 (ConvLSTM2D)  (None, 84, 37, 72, 32)    602240    
_________________________________________________________________
batch_normalization_1 (Batch (None, 84, 37, 72, 32)    128       
_________________________________________________________________
conv_lst_m2d_2 (ConvLSTM2D)  (None, 84, 37, 72, 32)    401536    
_________________________________________________________________
batch_normalization_2 (Batch (None, 84, 37, 72, 32)    128       
_________________________________________________________________
conv_lst_m2d_3 (ConvLSTM2D)  (None, 37, 72, 32)        4